# **Sezione riservata a Google Colab**

In [1]:
project_folder = './drive/MyDrive/Colab_Notebooks/Progetto_Finale/'

# **Libreria**

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


class ImageDataset:

    def __init__(self, ds_name, train_test, shuffle_files=True, data_dir="./data", USPS_dir="./USPS/"):

        if ds_name == 'mnist':

            ds = tfds.load('mnist', split=train_test, shuffle_files=shuffle_files,
                                download=True, data_dir=data_dir, as_supervised=True, batch_size=-1)

        elif ds_name == 'fmnist':

            ds = tfds.load('fashion_mnist', split=train_test, shuffle_files=shuffle_files,
                                download=True, data_dir=data_dir, as_supervised=True, batch_size=-1)

        elif ds_name == 'cifar10':

            ds = tfds.load('cifar10', split=train_test, shuffle_files=shuffle_files,
                                download=True, data_dir=data_dir, as_supervised=True, batch_size=-1)

        elif ds_name == 'kmnist':

            ds = tfds.load('kmnist', split=train_test, shuffle_files=shuffle_files,
                                download=True, data_dir=data_dir, as_supervised=True, batch_size=-1)

        elif ds_name == 'emnist':

            ds = tfds.load('emnist', split=train_test, shuffle_files=shuffle_files,
                                download=True, data_dir=data_dir, as_supervised=True, batch_size=-1)

        elif ds_name == 'svhn':

            ds = tfds.load('svhn', split=train_test, shuffle_files=shuffle_files,
                                download=True, data_dir=data_dir, as_supervised=True, batch_size=-1)

        elif ds_name == 'usps':

            self.images = np.load(USPS_dir + train_test + '_x.npy')
            self.labels = np.load(USPS_dir + train_test + '_y.npy')

        else:
            raise Exception("Selected dataset is not available")

        if ds_name != 'usps':
            self.images, self.labels = tfds.as_numpy(ds)

        self.num_channels = self.images.shape[3]

    def shuffle(self):

        p = np.random.permutation(len(self.images))
        self.images = self.images[p]
        self.labels = self.labels[p]


    def normalize(self, n_bits=8):

        self.images = self.images / (2**n_bits - 1)

    def filter(self, t1, t2=None, binary=True, overwrite=False):

        images = self.images
        labels = self.labels
        target_1 = np.where(labels == t1)
        target_2 = np.where(labels == t2)

        if t2 is not None:
            images = images[np.where((labels == t1) | (labels == t2))]
            labels = labels[np.where((labels == t1) | (labels == t2))]

        target_1 = np.where(labels == t1)
        target_2 = np.where(labels == t2)

        if binary:
            labels[target_1] = 1
            labels[target_2] = 0

        if not overwrite:
          return images, labels
          
        else:
          self.images = images
          self.labels = labels

    def pad(self, target_shape=(32, 32)):
        """
        Adds symmetric zero padding to image with shape HxWxC
        :param target_shape: tuple with target shape (H,W)
        """

        self.images = np.pad(self.images, ((0,0),(2,2),(2,2),(0,0)))

    def vectorize(self, merge_channels=False):
        """
        Transforms image from pixel matrix with shape (H,W,C) to linear vector
        :param merge_channels: if True, the pixel vectors from each channel will be concatenated
        """

        self.images = np.transpose(self.images, (0, 3, 1, 2))

        if merge_channels:
            self.images = np.reshape(self.images, [len(self.images), -1])

        else:
            self.images = np.reshape(self.images, [len(self.images), self.images.shape[1], -1])

    def permute(self, n):

        perm_idx = get_permutation(n)
        if self.num_channels == 3:
            perm_idx = np.concatenate((perm_idx,perm_idx,perm_idx),0)
        self.images = self.images[:, perm_idx]

    def subset(self, shard=False, shard_number=None, validation=False, validation_size=None, tile=None):

        if validation and validation_size is None:
            raise Exception("Requested training/validation split but no validation split size given.")
        if shard and shard_number is None:
            raise Exception("Requested sharding but no shard size given.")

        if validation:
            valid_images, valid_labels = self.images[0:validation_size], self.labels[0:validation_size]
            train_images, train_labels = self.images[validation_size:], self.labels[validation_size:]
            if tile is not None:
                valid_images = np.tile(valid_images, (1, tile))
                train_images = np.tile(train_images, (1, tile))
            if shard:
                valid_images, valid_labels = np.array_split(valid_images, shard_number), np.array_split(valid_labels, shard_number)
                train_images, train_labels = np.array_split(train_images, shard_number), np.array_split(train_labels, shard_number)
            return train_images, train_labels, valid_images, valid_labels
        else:
            train_images, train_labels = self.images, self.labels
            if tile is not None:
                train_images = np.tile(train_images, (1, tile))
            if shard:
                train_images, train_labels = np.array_split(train_images, shard_number), np.array_split(train_labels, shard_number)
            return train_images, train_labels


def get_matrix(n):
    '''
     Assumes that the matrix is of size 2^n x 2^n for some n

     EXAMPLE for n=4

     Old order:

      1  2  3  4
      5  6  7  8
      9 10 11 12
     13 14 15 16

     New order:

      1  2  5  6
      3  4  7  8
      9 10 13 14
     11 12 15 16

     Function returns numbers from old order, read in the order of the new numbers:

     [1, 2, 5, 6, 3, 4, 7, 8, 9, 10, 13, 14, 11, 12, 15, 16]

     So if you previously had a data vector v from a matrix size 32 x 32,
     you can now use v[get_permutation(5)] to reorder the elements.
    '''
    if n == 0:
        return np.array([[1]])
    else:
        smaller = get_matrix(n - 1)
        num_in_smaller = 2 ** (2 * n - 2)
        first_stack = np.hstack((smaller, smaller + num_in_smaller))
        return np.vstack((first_stack, first_stack + 2 * num_in_smaller))


def get_permutation(n):
    return get_matrix(n).ravel() - 1


# **Body**

In [3]:
import tensorflow as tf

class treeLayer(tf.keras.layers.Layer):

  def __init__(self, Input_size=3072, Activation="relu"):

    super(treeLayer, self).__init__()
    self.Input_size = Input_size
    self.Activation = Activation

  def build(self, input_shape):

    self.kernel = self.add_weight(shape=(1, self.Input_size),
                                  initializer=tf.keras.initializers.HeNormal,
                                  trainable=True)
    self.summer = np.zeros((self.Input_size, self.Input_size//2))
    for i in range(self.Input_size):
        self.summer[i, i//2] = 1

    self.summer = tf.convert_to_tensor(self.summer, dtype=tf.float32)
      
  def call(self, inputs):  
    
    x = tf.math.multiply(inputs, self.kernel)
    x = tf.matmul(x, self.summer)
    x = tf.nn.leaky_relu(x, alpha = .01)
    return x

def create_model(input_size, num_trees=1):
  model = tf.keras.Sequential()
  while input_size > num_trees:
    model.add(treeLayer(input_size))
    input_size = input_size//2
  model.add(tf.keras.layers.Dense(units=1, activation='sigmoid', use_bias=False))

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(
                      reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE
                  ),
                  metrics=['acc'])
  
  return model


In [ ]:
import gc

# Initialize settings
bs = 256
trials = 10
epochs = 2000
trees_set = [1,2,4,8,16,32]

# Load class-dataset list
# classes = np.load(project_folder + 'results/classes.npy', allow_pickle=True)

classes = [[3, 5, 'mnist']]

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=60)

history = np.load(project_folder + 'results/fcnn_history.npy', allow_pickle=True)
#history = np.zeros((len(classes), trials, len(trees_set), 2))

# For each dataset enumerated from classes list
for j, (t1, t2, ds) in enumerate(classes):

    print(f"Dataset: {ds} / Pair: {t1}-{t2}")

    test_ds = ImageDataset(ds, 'test')
    train_ds = ImageDataset(ds, 'train')

    for x in [train_ds, test_ds]:
        x.filter(t1, t2, overwrite=True)
        x.shuffle()
        x.normalize()
        if x.images.shape[1:3] == (28, 28):
            x.pad()
        x.vectorize(True)
    

    for k, trees in enumerate(trees_set):

      print(f"{trees}-FCNN")
      X_train, y_train, X_valid, y_valid = train_ds.subset(shard=True, shard_number=trials, validation=True,
                                                           validation_size=len(test_ds.images))
      X_test, y_test = test_ds.subset()
      test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

      if history[j, -1, k, 0] != 0:
        continue

      for i in range(trials):

        print(f"Trial {i+1}")

        #with tf.device('/device:GPU:0'):

        model = create_model(input_size=X_train[i].shape[1]*trees, num_trees=trees)

        train_set = tf.data.Dataset.from_tensor_slices((X_train[i], y_train[i])).map(lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)
        valid_set = tf.data.Dataset.from_tensor_slices((X_valid[i], y_valid[i])).map(lambda x, y: (tf.tile(x, [trees]), y)).batch(bs)

        fit_history = model.fit(x = train_set, batch_size = bs, epochs = epochs,
                      validation_data = valid_set, validation_batch_size = bs,
                      callbacks=[callback], verbose=0)
                  
        n_epochs = len(fit_history.history['acc'])
        train_loss = round(fit_history.history['loss'][-1]*100, 2)
        train_acc = round(fit_history.history['acc'][-1]*100, 2)
        valid_loss = round(fit_history.history['val_loss'][-1]*100, 2)
        valid_acc = round(fit_history.history['val_acc'][-1]*100, 2)
        print(f"Epochs: {n_epochs}/{epochs} - Train loss: {train_loss}%, accuracy: {train_acc}% - Validation loss: {valid_loss}%, accuracy: {valid_acc}%")

        evaluate_history = model.evaluate(x = test_set, batch_size = bs, verbose=0)
        print(f"Test loss: {round(evaluate_history[0]*100, 2)}%, accuracy: {round(evaluate_history[1]*100, 2)}%")
        
        history[j, i, k] = evaluate_history

        np.save(project_folder+'results/fcnn_history.npy', history,
                allow_pickle=True)
        
        del model, train_set, valid_set
        gc.collect()

Dataset: mnist / Pair: 3-5


local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to ./data/mnist/3.0.1. Subsequent calls will reuse this data.
Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


Instructions for updating:
Use `tf.data.Dataset.get_single_element()`.


1-FCNN
2-FCNN
4-FCNN
8-FCNN
16-FCNN
32-FCNN
Trial 1


In [ ]:
print(history)
model.summary()